In [1]:
!pip3 install -q google-generativeai
# !pip3 install -q pymongo

In [2]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

# Set up the API key
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Choose a model
model = genai.GenerativeModel("gemini-2.0-flash")

# Ask a question
question = "What is the capital of France?"
response = model.generate_content(question)

# Print the answer
print(response.text)


The capital of France is Paris.



In [5]:
# MongoDB Integration
import os
from dotenv import load_dotenv
from pymongo import MongoClient

# Load environment variables from .env file
load_dotenv()

# Get credentials
username = os.getenv("MONGODB_USERNAME")
password = os.getenv("MONGODB_PASSWORD")

# Construct URI
uri = (
    f"mongodb+srv://{username}:{password}"
    "@cluster0.ph7revd.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"
)

# Initialize client
client = MongoClient(
    uri,
    tls=True,
    tlsAllowInvalidCertificates=True,
    tlsAllowInvalidHostnames=True,
)

# Test connection
try:
    client.admin.command("ping")
    print("✅ Successfully connected to MongoDB!")
except Exception as e:
    print(f"Failed to connect to MongoDB: {e}")


✅ Successfully connected to MongoDB!


In [4]:
import requests
from datetime import datetime, timedelta, timezone
import os
from dotenv import load_dotenv

load_dotenv()

# Set your GitHub token and repo info
token = os.getenv("GITHUB_TOKEN") 
owner = "CSin007"
author = "Arushitib"
repo = "Bitcamp2025"

# Get time 24 hours ago
since = (datetime.now(timezone.utc) - timedelta(days=1)).isoformat()
headers = {"Authorization": f"token {token}"}
url = f"https://api.github.com/repos/{owner}/{repo}/commits"
params = {"since": since, "author": author}

# response = requests.get(url, headers=headers, params=params)
commits = requests.get(url, headers=headers, params=params).json()

print(f"Commits in last 24 hours: {len(commits)}")

total_additions = 0
total_deletions = 0

for commit in commits:
    sha = commit["sha"]
    commit_url = f"https://api.github.com/repos/{owner}/{repo}/commits/{sha}"
    commit_data = requests.get(commit_url, headers=headers).json()

    for file in commit_data.get("files", []):
        total_additions += file.get("additions", 0)
        total_deletions += file.get("deletions", 0)

print(f"Commits: {len(commits)}")
print(f"Lines Added: {total_additions}")
print(f"Lines Removed: {total_deletions}")


Commits in last 24 hours: 12
Commits: 12
Lines Added: 18896
Lines Removed: 213


In [6]:
# 

import os
import base64
import requests

CLIENT_ID = os.getenv("FITBIT_CLIENT_ID")
CLIENT_SECRET = os.getenv("FITBIT_CLIENT_SECRET")
AUTH_CODE = ""
REDIRECT_URI = "http://localhost:8080/callback"  

b64_auth = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()

headers = {
    "Authorization": f"Basic {b64_auth}",
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {
    "client_id": CLIENT_ID,
    "grant_type": "authorization_code",
    "redirect_uri": REDIRECT_URI,
    "code": AUTH_CODE,
}

res = requests.post("https://api.fitbit.com/oauth2/token", headers=headers, data=data)
print(res.json())


{'errors': [{'errorType': 'invalid_request', 'message': 'Missing parameters: code Visit https://dev.fitbit.com/docs/oauth2 for more information on the Fitbit Web API authorization process.'}], 'success': False}


In [15]:
def calculate_burnout(steps, minutes_asleep, commits, fatigue_score):
    """_summary_

    Args:
        steps (_list_): array of int values of steps each day for last 5 days
        minutes_asleep (_list_): array of int values of mins slept each day for last 5 days
        commits (_list_): array of int values of commits made each day for last 5 days
        fatigue_score (_float_): score from 1-10 of how fatigued you feel

    Returns:
        _float_: returns a burnout score from 0-100
    """

    def normalize(val, min_val, max_val):
        return (val - min_val) / (max_val - min_val)

    def avg(arr):
        return sum(arr) / len(arr)

    # Normalize features to [0, 1]
    sleep_norm = 1 - normalize(avg(minutes_asleep), 360, 540)  # ideal: 6-9 hours
    steps_norm = 1 - normalize(avg(steps), 5000, 10000)  # ideal: 5k–10k steps
    commits_norm = normalize(avg(commits), 0, 15)  # high commit count = higher burnout
    fatigue_norm = normalize(fatigue_score, 1, 10)  # already 1-10

    # Weighted burnout score
    burnout = (
        0.40 * sleep_norm
        + 0.20 * steps_norm
        + 0.20 * commits_norm
        + 0.20 * fatigue_norm
    )

    return round(burnout * 100, 2)

minutes_asleep = [347, 445, 388, 81, 345]
steps = [14463, 13081, 9265, 169, 13060]
commits = [13, 0, 0, 0, 0]
fatigue_score = 8.7

calculate_burnout(steps, minutes_asleep, commits, fatigue_score)

69.17